Import librairies

In [ ]:
import gspread
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from oauth2client.service_account import ServiceAccountCredentials
from matplotlib import rcParams
from utils.sheets_to_dataframe import sheets_to_df
rcParams['font.family'] = 'MS Gothic'

%load_ext autoreload
%autoreload 2

Connection with google sheets

In [9]:
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('../api-key.json', scope)
client = gspread.authorize(creds)

# Open Google Sheets
spreadsheet = client.open_by_key('1tWzHyOUIsNMERChwDtMJlgUURB0aIxNlOkPDp5CDTkk')

Get every tables

In [51]:
data_sheet = spreadsheet.get_worksheet(1)
hocg_analysis_sheet = spreadsheet.get_worksheet(2)
general_data_sheet = spreadsheet.get_worksheet(5)

data_table = data_sheet.get_all_records()
market_value_table = hocg_analysis_sheet.get('A1:D2')
product_relative_data_table = hocg_analysis_sheet.get('A5:L19')
association_table = hocg_analysis_sheet.get('A23:CN36')
collabs_table = hocg_analysis_sheet.get('A38:C39')
boosters_general_table = general_data_sheet.get('A1:D3')
decks_general_table = general_data_sheet.get('A5:H7')
oshi_association_table = general_data_sheet.get('A9:BW10')

Generate data frames

In [ ]:
df_data = pd.DataFrame(data_table)

df_market_value = sheets_to_df(market_value_table, df_type="numeric")

for i in range(len(product_relative_data_table[0]) - len(product_relative_data_table[1])):
    for row in product_relative_data_table[1:]:
        row.append(np.nan)
df_product_relative_data = sheets_to_df(product_relative_data_table, mode="rows", df_type="numeric")

df_association = sheets_to_df(association_table, mode="rows", df_type="numeric")

df_collabs = sheets_to_df(collabs_table, mode="rows", df_type="numeric")

df_boosters_general = sheets_to_df(boosters_general_table, transpose=True)

df_decks_general = sheets_to_df(decks_general_table, transpose=True)

df_oshi_association = sheets_to_df(oshi_association_table, mode="rows")

In [ ]:
dates_list = [date.strftime('%Y年%m月%d日').replace(f"0{date.month}月", f"{date.month}月").replace(f"0{date.day}日", f"{date.day}日") for date in sorted(pd.to_datetime(list(set(df_boosters_general['Release date']).union(set(df_decks_general['Release date']))), format='%Y年%m月%d日'))]

new_df = pd.DataFrame(columns=dates_list, index=["Boosters", "Decks"])

new_df.loc["Boosters"] = {date: {"names": "", "nb cards": 0} for date in dates_list}
new_df.loc["Decks"] = {date: {"names": "", "nb cards": 0} for date in dates_list}

for index, row in df_boosters_general.iterrows():
    current_names = new_df[row['Release date']]["Boosters"]["names"]
    new_df[row['Release date']]["Boosters"]["names"] = "\n".join([current_names, row['Boosters Names']]) if current_names else row['Boosters Names']
    new_df[row['Release date']]["Boosters"]["nb cards"] += df_product_relative_data[row['Boosters Names']]["Total cards by products*"]

for index, row in df_decks_general.iterrows():
    current_names = new_df[row['Release date']]["Decks"]["names"]
    new_df[row['Release date']]["Decks"]["names"] = "\n".join([current_names, row['Decks Names']]) if current_names else row['Decks Names']
    new_df[row['Release date']]["Decks"]["nb cards"] += df_product_relative_data[row['Decks Names']]["Total cards by products*"]

boosters_values = [np.nan if value["nb cards"] == 0 else value["nb cards"] for value in new_df.loc["Boosters"].values]
decks_values = [np.nan if value["nb cards"] == 0 else value["nb cards"] for value in new_df.loc["Decks"].values]
boosters_values[3] = 0
decks_values[2] = 0
boosters_values = np.interp([0, 1, 2, 3], [0, 1, 3], [boosters_values[0], boosters_values[1], boosters_values[3]]) 

plt.plot(new_df.columns, boosters_values, marker='o', label="Ligne 1", color='blue', linestyle='-', markevery=[0, 1, 3], linewidth=2)
plt.plot(new_df.columns, decks_values, marker='o', label="Ligne 2", color='red', linestyle='-', markevery=[0, 1, 2], linewidth=2)

for index, (x, y) in enumerate(zip(new_df.columns, boosters_values)):
    if new_df[x]["Boosters"]["names"] != "":
        plt.text(x, y + 5, f"{new_df[x]["Boosters"]["names"]}\n{y}", ha='center', fontsize=8, color='black')

for index, (x, y) in enumerate(zip(new_df.columns, decks_values)):
    if new_df[x]["Decks"]["names"] != "":
        plt.text(x, y + 5, f"{new_df[x]["Decks"]["names"]}\n{y}", ha='center', fontsize=8, color='black')

plt.show()